In [79]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [80]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [81]:
len(words)

32033

In [82]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [83]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:

	print(w)
	context = [0] * block_size
	for ch in w + '.':
		ix = stoi[ch]
		X.append(context)
		Y.append(ix)
		print(''.join(itos[i] for i in context), '--->', itos[ix])
		context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [84]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [85]:
C = torch.randn((27, 2))

In [86]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [87]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [88]:
# emb @ W1 + b1
# this won't work because emb needs to be 32 x 6

In [89]:
# this will work 
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h

tensor([[ 0.5047,  0.9347, -0.4626,  ...,  0.6712, -0.5863, -0.9869],
        [ 0.0387,  0.9845,  0.2227,  ...,  0.5614, -0.8264, -0.9938],
        [ 0.6057,  0.8218, -0.6775,  ..., -0.4522, -0.9627, -0.9958],
        ...,
        [-0.9995, -0.9781,  0.8896,  ..., -0.9989, -0.6631,  0.9892],
        [-0.7354, -1.0000, -0.9788,  ..., -1.0000,  0.8256,  0.9530],
        [-0.9985, -0.9632,  0.9886,  ...,  0.3313,  1.0000,  0.9999]])

In [90]:
h.shape

torch.Size([32, 100])

In [91]:
(emb.view(-1, 6) @ W1).shape

torch.Size([32, 100])

In [92]:
b1.shape

torch.Size([100])

In [93]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [94]:
logits = h @ W2 + b2

In [95]:
logits.shape

torch.Size([32, 27])

In [96]:
logits

tensor([[ 1.4327e+01, -4.3909e+00, -6.5111e+00, -9.0079e-02,  1.8499e+01,
          1.4099e+01,  5.7481e+00,  3.8833e+00, -3.2791e+00,  2.9106e+00,
          2.3689e+00, -1.4108e+01,  1.7157e+00,  9.2348e+00,  3.4168e+00,
         -1.4659e+01,  2.2924e+00, -2.8023e+00, -5.6702e+00, -2.0260e+01,
         -2.6559e+00,  4.3171e+00,  7.2855e+00, -5.1402e+00,  2.3521e-01,
          1.5005e+01,  4.7538e+00],
        [ 1.3022e+01, -6.0755e+00, -6.9659e+00, -3.8214e+00,  1.7740e+01,
          1.3330e+01,  5.2424e+00,  3.2346e+00, -7.9820e-01,  4.9643e+00,
          2.5765e+00, -1.3549e+01,  2.4546e+00,  6.4673e+00,  1.5239e+00,
         -9.6305e+00,  1.8227e-01, -3.1334e-01, -1.8127e+00, -2.2599e+01,
         -1.2306e+00,  3.2419e+00,  7.8294e+00, -5.1530e+00,  1.4556e+00,
          9.3549e+00,  3.6583e+00],
        [ 1.2299e+01, -2.1867e+00, -6.1252e+00, -2.9940e+00,  1.9291e+01,
          1.6285e+01,  3.0073e+00, -2.2641e+00, -4.4728e+00,  5.4717e-01,
          9.5768e-01, -1.6817e+01,  2.98

In [97]:
counts = logits.exp()

In [98]:
prob = counts / counts.sum(1, keepdims=True)

In [99]:
prob.shape

torch.Size([32, 27])

In [100]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(19.4594)

In [101]:
# ------------- now made respectable -------------

In [102]:
# dataset
X.shape, Y.shape 

(torch.Size([32, 3]), torch.Size([32]))

In [103]:
C = torch.randn((27, 2))

W1 = torch.randn((6, 100))
b1 = torch.randn(100)

W2 = torch.randn((100, 27))
b2 = torch.randn(27)

parameters = [C, W1, b1, W2, b2]

In [104]:
sum(p.nelement() for p in parameters) # number of parameters in total

3481

In [105]:
emb = C[X] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
# coutns = logits.exp()
# prob = counts / counts.sum(1, keepdims=True)
# loss = -prob[torch.arange(32), Y].log().mean()
loss = F.cross_entropy(logits, Y)
loss

tensor(18.7446)